In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.utils import resample,shuffle
import csv
import random
import math
import operator

In [13]:
def tfpn(testSet, predictions):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            if(testSet[x][classcol] == 1):
                tp+=1
            else:
                tn+=1
        else:
            if(predictions[x] ==1):
                fp+=1
            else:
                fn+=1
    return (tp,tn,fp,fn)
def findclass(df):
    classs=None
    columns=df.columns.tolist()
    for i in range(len(columns)):
        if columns[i]=='class':
            classs=i
            break
    return classs
classcol=None
def loaddataset(file,perc,trainset=[],testset=[]):
    data=pd.read_csv(file)
    df_maj = data[data['class'] == 1]
    l = len(df_maj)
    #print(len(df_maj))
    df_min = data[data['class'] == 0]
    df_minor_up = resample(df_min,replace=True,n_samples=l,random_state=123)
    #print(len(df_minor_up))
    df_upsampled = pd.concat([df_maj,df_minor_up])
    df_upsampled=shuffle(df_upsampled)
    df=df_upsampled.drop(['spectrometric_redshift','Unnamed: 0', 'galex_objid', 'sdss_objid','pred'],1)
    global classcol 
    classcol=findclass(df)
    df=df[1:]
    dataset=list(df.values.tolist())
    length=len(dataset[0])
    count=0
    split=(len(dataset)-1)*perc
    for x in range(1,len(dataset)-1):
        for y in range(1,length):
            dataset[x][y]=float(dataset[x][y])
        if(count<split):
            trainset.append(dataset[x])
                #print(dataset[x])
        else:
            testset.append(dataset[x])
        count+=1
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow(float(instance1[x]) - float(instance2[x]), 2)
    return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)
    for x in range(1,len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][classcol]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            correct += 1
    print(len(testSet))
    return (correct/float(len(testSet))) * 100.0
def main(k):
# prepare data
    trainingSet=[]
    testSet=[]
    split = 0.7
    loaddataset('cat1.csv',split,trainingSet,testSet)
    # generate predictions
    predictions=[]
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        #print('predicted=' + str(result) + ', actual=' + str(testSet[x][classcol]))
    accuracy = getAccuracy(testSet, predictions)
    l = tfpn(testSet,predictions)
    TP = l[0]
    TN = l[1]
    FP = l[2]
    FN = l[3]

    print("true positive :",TP)
    print("true negative :",TN)
    print("false positive :",FP)
    print("false negative :",FN)
    TPR = TP/(TP+FN)
    print( 'Sensitivity:{}'.format(TPR))
    TNR = TN/(TN+FP)
    print( 'Specificity:{}'.format(TNR))
    Precision = TP/(TP+FP)
    print( 'Precision:{}'.format(Precision))
    Recall = TP/(TP+FN)
    print( 'Recall:{}'.format(Recall))
    Acc = (TP+TN)/(TP+TN+FP+FN)
    print( 'Accuracy:{}'.format(Acc))
    Fscore = 2*(Precision*Recall)/(Precision+Recall)
    print( 'FScore:{}'.format(Fscore))
main(3)

355
true positive : 182
true negative : 171
false positive : 0
false negative : 2
Sensitivity:0.9891304347826086
Specificity:1.0
Precision:1.0
Recall:0.9891304347826086
Accuracy:0.9943661971830986
FScore:0.994535519125683


In [14]:
main(7)

355
true positive : 171
true negative : 173
false positive : 0
false negative : 11
Sensitivity:0.9395604395604396
Specificity:1.0
Precision:1.0
Recall:0.9395604395604396
Accuracy:0.9690140845070423
FScore:0.9688385269121813


In [15]:
main(10)

355
true positive : 170
true negative : 174
false positive : 6
false negative : 5
Sensitivity:0.9714285714285714
Specificity:0.9666666666666667
Precision:0.9659090909090909
Recall:0.9714285714285714
Accuracy:0.9690140845070423
FScore:0.9686609686609686
